In [1]:
r = lambda x, y: np.sqrt(x**2 + y**2)

In [2]:
R = lambda x, y, z: np.sqrt(x**2 + y**2 + z**2)

In [5]:
def delta_sigma_point_Boussinesq(x, y, z, F, nu):
    Rad = R(x,y,z)
    rad = r(x,y)
    a = (3*F)/(2*np.pi*Rad**2)
    delta_sigma_xx = a * ( (x**2 * z)/(Rad**3) - (1 - 2*nu)/3 * ( (x**2 - y**2) * Rad/(rad**2 * (Rad + z)) + (y**2 * z)/(Rad * rad**2) ))
    delta_sigma_yy = a * ( (y**2 * z)/(Rad**3) - (1 - 2*nu)/3 * ( (x**2 - y**2) * Rad/(rad**2 * (Rad + z)) + (x**2 * z)/(Rad * rad**2) ))
    delta_sigma_zz = a * z**3/Rad**3
    delta_sigma_xy = a * ( (x * y * z)/(Rad**3) - (1 - 2*nu)/3 * (x * y * (2 * Rad + z))/(Rad * (Rad + z)**2) )
    delta_sigma_yz = a * (y * z**2)/(Rad**3)
    delta_sigma_xz = a * (x * z**2)/(Rad**3)
    return delta_sigma_xx, delta_sigma_yy, delta_sigma_zz, delta_sigma_xy, delta_sigma_yz, delta_sigma_xz

In [6]:
def delta_sigma_point_Boussinesq_dl(x, y, z, nu):
    Rad = R(x,y,z)
    rad = r(x,y)
    delta_sigma_xx = ( (x**2 * z)/(Rad**3) - (1 - 2*nu)/3 * ( (x**2 - y**2) * Rad/(rad**2 * (Rad + z)) + (y**2 * z)/(Rad * rad**2) ))
    delta_sigma_yy = ( (y**2 * z)/(Rad**3) - (1 - 2*nu)/3 * ( (x**2 - y**2) * Rad/(rad**2 * (Rad + z)) + (x**2 * z)/(Rad * rad**2) ))
    delta_sigma_zz = z**3/Rad**3
    delta_sigma_xy = ( (x * y * z)/(Rad**3) - (1 - 2*nu)/3 * (x * y * (2 * Rad + z))/(Rad * (Rad + z)**2) )
    delta_sigma_yz = (y * z**2)/(Rad**3)
    delta_sigma_xz = (x * z**2)/(Rad**3)
    return delta_sigma_xx, delta_sigma_yy, delta_sigma_zz, delta_sigma_xy, delta_sigma_yz, delta_sigma_xz

In [7]:
def plot_szz_Boussinesq():
    z = np.array([1,2,5,10]) # m
    F = 100 # kN
    y = 0. # nur in x-z Ebene (bzw. radial)
    x = np.linspace(-10,10,1000)

    fig, ax = plt.subplots(figsize=(12,6))
    for i in z:
        ax.plot(x,delta_sigma_point_Boussinesq(x,y,i,F,0.3)[2],label="$z = %i$ m" %i)

    ax.legend()
    ax.set_xlabel('$x$ / m')
    ax.set_ylabel('$\\Delta \\sigma_{zz}$ / kPa')
    fig.tight_layout();

In [8]:
def plot_szz_Boussinesq_depth():
    z = np.array([0.5,1,2,3,5]) # m
    y = 0. # nur in x-z Ebene (bzw. radial)
    x = np.linspace(-10,10,1000)

    fig, ax = plt.subplots(figsize=(12,6))
    for i in z:
        ax.plot(x,-delta_sigma_point_Boussinesq_dl(x,y,i,0.3)[2]/R(x,y,i)**2-i,label="$z = %i$ m" %i)
        ax.axhline(-i,ls=':',color='black')

    #ax.axhline(0,ls='')
    ax.legend()
    ax.set_xlabel('$x$ / m')
    ax.set_ylabel('$\\Delta \\sigma_{zz}$ / $[(3F)/(2 \\pi)]$; $z$ / m')
    ax.set_yticks([-1,-2,-3,-4,-5])
    ax.set_yticklabels([1,2,3,4,5])
    #ax.spines['bottom'].set_position(('data', 0))
    #ax.xaxis.set_label_position('top') 
    #ax.xaxis.tick_top()
    fig.tight_layout();

In [9]:
def plot_all_Boussinesq(nu):
    z = 1
    F = 100 # kN
    y = 0. # nur in x-z Ebene (bzw. radial)
    x = np.linspace(-5,5,1000)

    fig, ax = plt.subplots(figsize=(12,6))
    ax.plot(x,delta_sigma_point_Boussinesq(x,y,z,F,nu)[0],label="$\\Delta \\sigma_{xx}$")
    ax.plot(x,delta_sigma_point_Boussinesq(x,y,z,F,nu)[1],label="$\\Delta \\sigma_{yy}$")
    ax.plot(x,delta_sigma_point_Boussinesq(x,y,z,F,nu)[2],label="$\\Delta \\sigma_{zz}$")
    ax.plot(x,delta_sigma_point_Boussinesq(x,y,z,F,nu)[3],label="$\\Delta \\sigma_{xy}$",ls='--')
    ax.plot(x,delta_sigma_point_Boussinesq(x,y,z,F,nu)[4],label="$\\Delta \\sigma_{yz}$",ls='--')
    ax.plot(x,delta_sigma_point_Boussinesq(x,y,z,F,nu)[5],label="$\\Delta \\sigma_{xz}$",ls='--')

    ax.legend()
    ax.set_xlabel('$x$ / m')
    ax.set_ylabel('$\\Delta \\sigma_{ij}$ / kPa')
    ax.spines['bottom'].set_position(('data', 0))
    fig.tight_layout();

In [13]:
def plot_isobars_zz_Boussinesq(nu=0.3):
    fig, ax = plt.subplots(figsize=(12,12))
    xs = np.linspace(-5,5,500)
    zs = np.linspace(0,5,250)
    sigmas = np.zeros((len(xs),len(zs)))
    for i,x in enumerate(xs):
        for j,z in enumerate(zs):
            sigmas[i,j] = delta_sigma_point_Boussinesq(x,0,z,100,nu)[2]
    X, Z = np.meshgrid(xs,zs)
    cp = plt.contour(X, -Z, sigmas.T,levels=[1,2,4,8,16,32,64,128,256,512])
    ax.clabel(cp, inline=True, 
              fontsize=12)
    ax.set_xlabel('$x$ / m')
    ax.set_ylabel('$z$ / m')
    ax.set_yticks([-1,-2,-3,-4,-5])
    ax.set_yticklabels([1,2,3,4,5])
    ax.spines['bottom'].set_position(('data', 0))
    ax.xaxis.set_label_position('top') 
    ax.xaxis.tick_top()
    ax.set(adjustable='box', aspect='equal')
    fig.tight_layout();

In [14]:
def calc_principal_stresses(s):
    eigenValues, eigenVectors = np.linalg.eig(s)
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return eigenValues, eigenVectors

In [15]:
def vector_to_tensor(vec):
    return np.array([[vec[0],vec[3],vec[5]],
                    [vec[3],vec[1],vec[4]],
                    [vec[5],vec[4],vec[2]]])

In [ ]:
def plot_isobars_principal_Boussinesq(nu=0.3):
    fig, ax = plt.subplots(figsize=(18,6),ncols=3)
    xs = np.linspace(-5,5,500)
    zs = np.linspace(0,5,250)
    s1 = np.zeros((len(xs),len(zs)))
    s2 = np.zeros((len(xs),len(zs)))
    s3 = np.zeros((len(xs),len(zs)))
    for i,x in enumerate(xs):
        for j,z in enumerate(zs):
            sigma_vec = delta_sigma_point_Boussinesq(x,0,z,100,nu)
            e_val, e_vec = calc_principal_stresses(vector_to_tensor(sigma_vec))
            s1[i,j] = e_val[0]
            s2[i,j] = e_val[1]
            s3[i,j] = e_val[2]
    X, Z = np.meshgrid(xs,zs)
    cp1 = ax[0].contour(X, -Z, s1.T,levels=[1,2,4,8,16,32,64,128,256,512])
    cp2 = ax[1].contour(X, -Z, s2.T,levels=[-8,-4,-2,-1,-0.2])
    cp3 = ax[2].contour(X, -Z, s3.T,levels=[-64,-32,-16,-8,-4,-2,-1,-0.3])
    ax[0].clabel(cp1, inline=True, 
              fontsize=12)
    ax[1].clabel(cp2, inline=True, 
              fontsize=12)
    ax[2].clabel(cp3, inline=True, 
              fontsize=12)
    for i in range(3):
        ax[i].set_xlabel('$x$ / m')
        ax[i].set_ylabel('$z$ / m')
        ax[i].set_yticks([-1,-2,-3,-4,-5])
        ax[i].set_yticklabels([1,2,3,4,5])
        ax[i].spines['bottom'].set_position(('data', 0))
        ax[i].xaxis.set_label_position('top') 
        ax[i].xaxis.tick_top()
        ax[i].set(adjustable='box', aspect='equal')
    fig.tight_layout();